In [1]:
import numpy as np
# from qiskit import QuantumCircuit, Aer, transpile, assemble
# from qiskit import *
# from qiskit.visualization import plot_histogram
from math import gcd, sqrt, pi, trunc, exp, sqrt, sinh, log, sin, cos, tan, atan
import cmath
# from numpy.random import randint
# import pandas as pd
# from scipy.linalg import expm, sinm, cosm
from fractions import Fraction
from numpy import linalg as LA
# from qutip import *
# from scipy import *
from scipy.linalg import norm
# from sympy import I, Matrix, symbols
# from sympy.physics.quantum import TensorProduct
import time
# from datetime import datetime
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import random
import pickle
import os
from numba import jit, cuda
import gc

In [2]:
q = 3   # 2, 3, 4, ...
D = 2
ghost = 0
Size = 64

J = 1
Tc = 1.005
# Tc = 1 / log(1 + sqrt(q))

# Export = [True, False, False]  # Normal / Correlation Function / MCRG
Export = [False, True, False]  # Normal / Correlation Function / MCRG
# Export = [True, True, False]  # Normal / Correlation Function / MCRG

Ensemble = 64
Iteration = 2048
# Collect = 8192
Collect = 128

TRound = 3
HRound = 4
DataRound = 3

Initial = 'Uniform'

print(Size, Tc)

64 1.005


In [3]:
Coordinate_List = [[] for i in range(pow(Size, D))]
NN_Coordinate = [[] for i in range(pow(Size, D))]

for i in range(pow(Size, D)):
    for j in range(D):
        Coordinate_List[i].append(int((i % pow(Size, j + 1)) / pow(Size, j)))

for i in range(pow(Size, D)):
    for j in range(D):
        NN_Coordinate[i].append(i + ((Coordinate_List[i][j] + 1) % Size - Coordinate_List[i][j]) * pow(Size, j))
        NN_Coordinate[i].append(i + ((Coordinate_List[i][j] + Size - 1) % Size - Coordinate_List[i][j]) * pow(Size, j))

print(NN_Coordinate[0])

Distance_List = [[0 for i in range(pow(Size, D))] for i in range(pow(Size, D))]
Irreducible_Distance = [0.0]

for i in range(pow(Size, D)):
    Distance_List[i][i] = 0.0
        
    for j in range(i + 1, pow(Size, D)):
        for k in range(D):
            Distance_List[i][j] += pow(min(abs(Coordinate_List[i][k] - Coordinate_List[j][k]), Size - abs(Coordinate_List[i][k] - Coordinate_List[j][k])), 2)
        
        Distance_List[i][j] = np.sqrt(Distance_List[i][j])
        Distance_List[j][i] = Distance_List[i][j]
        
        if(Distance_List[i][j] not in Irreducible_Distance):
            Irreducible_Distance.append(Distance_List[i][j])

test = 0
print(Distance_List[test][NN_Coordinate[test][1]])

Irreducible = len(Irreducible_Distance)
print(Irreducible)

[1, 63, 64, 4032]
1.0
457


In [4]:
DATA_LOCATION = '/mnt/d/Research/potts/data'
LOG_LOCATION = '/mnt/d/Research/potts/log'

File_Data = [os.path.join(DATA_LOCATION, 'data potts (normal).txt'), 
             os.path.join(DATA_LOCATION, 'data potts (corr).txt'), 
             os.path.join(DATA_LOCATION, 'data potts (mcrg).txt')]

File_Log = [os.path.join(LOG_LOCATION, 'log potts (normal).txt'), 
            os.path.join(LOG_LOCATION, 'log potts (corr).txt'), 
            os.path.join(LOG_LOCATION, 'log potts (mcrg).txt')]

def PrintFile(FILE, Data):
    if(np.size(Data) == len(Data)):            
        for i in range(len(Data)):
            FILE.write(str(Data[i]))
            FILE.write(" ")

        FILE.write("\n")
        
    else:
        for i in range(len(Data)):
            for j in range(len(Data[i])):
                FILE.write(str(Data[i][j]))
                FILE.write(" ")
            
            FILE.write("\n")
    
def PrintResult(Data, Time, Parameter, i):
    T, H = Parameter[0], Parameter[1]

    if(i == 0):
        return [q, D, Size, Ensemble, Iteration, Collect, Time, T, H, Data[0], Data[1], Data[2], Data[3], Data[4]]
        
    elif(i == 1):
        return [[q, D, Size, Ensemble, Iteration, Collect, Time, T, H, Irreducible_Distance[j], Data[j]] for j in range(Irreducible)]
        
    elif(i == 2):
        return [q, D, Size, Ensemble, Iteration, Collect, Time, T, H, Data[0], Data[1]]

def ExportData(Data, Parameter, Time):
    T, H = Parameter[0], Parameter[1]
    
    print('\n')
    
    for i in range(3):
        if(Export[i]):
            FILE = open(File_Data[i], 'a')
            LOG = open(File_Log[i], 'a')
            
            PrintFile(FILE, PrintResult(Data[i], Time, Parameter, i))
            PrintFile(LOG, [q, D, Size, Ensemble, Iteration, Collect, Time, T, H])

            print("Data exported to", File_Data[i])

            FILE.close()
            LOG.close()
        
        else:
            print("No Data exported to", File_Data[i])

In [5]:
Mode, Direction, Value = 'T', '+', [0.0001, 2, 3, 11]
# Mode, Direction, Value = 'H', '+', [0.0001, 2, 0, 12]
# Mode, Direction, Value = 'Critical', '', ''

def Initial_Condition(Mode, Direction, Value):
    Length = 1
        
    if(Mode != 'Critical'):
        Length = Value[3] - Value[2] + 1

    Coupling_Parameter = [[0 for i in range(2)] for i in range(Length)]

    if(Mode == 'Critical'):
        Coupling_Parameter[0][0] = Tc
        Coupling_Parameter[0][1] = 0
    
    else:    
        Min = Value[0]
        Base = Value[1]
        Start = Value[2]
        End = Value[3]

        if(Mode == 'T'):
            for i in range(Start, End + 1):
                if(Direction == '+'):
                    Coupling_Parameter[i - Start][0] = Tc + Min * pow(Base, i)
                    Coupling_Parameter[i - Start][1] = 0
            
                elif(Direction == '-'):
                    Coupling_Parameter[i - Start][0] = Tc - Min * pow(Base, i)
                    Coupling_Parameter[i - Start][1] = 0

        elif(Mode == 'H'):
            for i in range(Start, End + 1):
                if(Direction == '+'):
                    Coupling_Parameter[i - Start][0] = Tc
                    Coupling_Parameter[i - Start][1] = Min * pow(Base, i)
                
                elif(Direction == '-'):
                    Coupling_Parameter[i - Start][0] = Tc
                    Coupling_Parameter[i - Start][1] = - Min * pow(Base, i)
    
    return Coupling_Parameter

Coupling_Parameter = Initial_Condition(Mode, Direction, Value)

In [6]:
def Array_Data(Parameter):
    Array = Initial_State()

    Temp = [[] for _ in range(pow(Size, D))]
    
    for _ in range(Iteration):
        Metropolis(Array, Parameter)
    
    for _ in range(Collect):
        Metropolis(Array, Parameter)
        
        for i in range(pow(Size, D)):
            Temp[i].append(Array[i])
    
    del Array
    
    return Temp

def Metropolis(Array, Parameter):
    for _ in range(pow(Size, D)):
        x = random.randrange(0, pow(Size, D))
        p = random.random()

        Probability = Partition_Function(Array, Parameter, x)
        
        for i in range(q):     
            if(p >= Probability[i] and p <= Probability[i + 1]):
                Array[x] = i
        
        del x, p, Probability

def Partition_Function(Array, Parameter, x):
    T, H = Parameter[0], Parameter[1]
    
    Probability = [0]
    Norm = 0

    for i in range(q):
        Neighbor = - H * ddf(i, ghost)
    
        for j in range(len(NN_Coordinate[x])):      
            Neighbor += - J * ddf(i, Array[NN_Coordinate[x][j]])

        Probability.append(exp(- Neighbor / T))

        Norm += Probability[i + 1]

    for i in range(q):
        Probability[i + 1] = Probability[i] + Probability[i + 1] / Norm
    
    del T, H, Norm
    
    return Probability

def Initial_State():
    if(Initial == 'Random'):    
        return [random.randrange(0, q) for i in range(pow(Size, D))]
    
    elif(Initial == 'Uniform'):
        return [ghost for i in range(pow(Size, D))]

def ddf(a, b):
    if(a == b):
        return 1
    else:
        return 0

In [7]:
def Order_Parameter(Array):
    num = 0

    for i in range(pow(Size, D)):
        num += ddf(Array[i], ghost)

    return (q * num / pow(Size, D) - 1) / (q - 1)

def Hamiltonian(Array, Parameter):
    H = Parameter[1]
    
    Energy = 0

    for i in range(pow(Size, D)):
        Energy += - H * ddf(Array[i], ghost) / pow(Size, D)

        for j in range(len(NN_Coordinate[i])):
            Energy += - J * ddf(Array[i], Array[NN_Coordinate[i][j]]) / 2.0 / pow(Size, D)
    
    return Energy

def Transform(Temp):
    num = 0
    
    for i in range(len(Temp)):
        num += ddf(Temp[i], ghost)
    
    return num / len(Temp)

def Transform_Array(Temp1, Temp2):
    num = 0
    
    for i in range(len(Temp1)):
        num += ddf(Temp1[i], Temp2[i])
    
    return num / len(Temp1)

def Binder(Temp):
    square = 0
    quad = 0

    for i in range(len(Temp)):
        quad += pow(Temp[i], 4) / len(Temp)
        square += pow(Temp[i], 2) / len(Temp)
    
    return (quad / pow(square, 2))

def Record(Parameter):
    Temp = Array_Data(Parameter)
    
    Ensemble_Normal = [0 for i in range(5)] # Order Parameter, Susceptibility, Binder Cumulant, Energy, Specific Heat
    Ensemble_Corr = [0 for i in range(Irreducible)] # Correlation Function
    Ensemble_MCRG = [0 for i in range(2)] # yh, yt    
    
    if(Export[0]):
        Ensemble_Normal = Record_Normal(Temp, Parameter)    
    if(Export[1]):
        Ensemble_Corr = Record_Corr(Temp)        
    if(Export[2]):
        Ensemble_MCRG = Record_MCRG(Temp)
        
    del Temp
    gc.collect()
    
    return Ensemble_Normal, Ensemble_Corr, Ensemble_MCRG

def Record_Normal(Temp, Parameter):
    Temp = np.array(Temp)
    M, E = [], []
    
    for i in range(len(Temp[0])):
        M.append(Order_Parameter(Temp[:, i]))
        E.append(Hamiltonian(Temp[:, i], Parameter))
    
    return [np.average(M), pow(np.std(M), 2), 1 - Binder(M) / 3, np.average(E), pow(np.std(E), 2)] # Order Parameter

def Record_Corr(Temp):
    Irr = [[] for i in range(Irreducible)]
    Average = [0 for i in range(pow(Size, D))]

    for i in range(pow(Size, D)):
        Average[i] = (q * Transform(Temp[i]) - 1) / (q - 1)    
        Irr[Irreducible_Distance.index(Distance_List[i][i])].append((q * Transform_Array(Temp[i], Temp[i]) - 1) / (q - 1) - Average[i] * Average[i])
            
    for i in range(pow(Size, D)):
        for j in range(i + 1, pow(Size, D)):
            temp = (q * Transform_Array(Temp[i], Temp[j]) - 1) / (q - 1) - Average[i] * Average[j]
            dist = Irreducible_Distance.index(Distance_List[i][j])
            Irr[dist].append(temp)
            Irr[dist].append(temp)
            
            del temp, dist
    
    del Average    
    gc.collect()    
    
    return [np.average(Irr[i]) for i in range(Irreducible)]    

def Record_MCRG(Temp):
    S1, S2 = [], []
    
    for i in range(pow(Size, D)):
        S1.append(Transform(Temp[i]))
        
        for j in range(len(NN_Coordinate[i])):
            S2.append(Transform_Array(Temp[i], Temp[NN_Coordinate[i][j]]))
    
    return [np.sum(S1), np.sum(S2) / 2.0]

In [8]:
def Print_Array(Array):
    if(D == 2):
        Print = [[] for i in range(Size)]

        for i in range(Size):
            for j in range(Size):
                Print[i].append(Array[Size * i + j])

        for line in np.array(Print):
            print('  '.join(map(str, line)))
        
        print("\n")

def Ensemble_Print(num, Time):
    if(num % 8 == 0 and num != 0):
        print("\n", end = "") 

    if(num < 9):
        print(" ", num + 1, "(", Time, "s )", end = " ")
        
    elif(num >= 9):
        print("", num + 1, "(", Time, "s )", end = " ")

def Size_Print():
    for i in range(D - 1):
        print(Size, "x", end = " ")
    
    print(Size, ")")

def Condition_Print(Status):
    t = round(Tc, TRound)
    
    if(Status == -1):
        if(Mode == 'T'):
            print(q, "- state ", D, "D  Potts  Temperature :  H = 0, T = Tc ( =", t, ")", Direction, Value[0], "X", Value[1], "^ (", Value[2], "~", Value[3], ')')
        
        elif(Mode == 'H'):
            print(q, "- state ", D, "D  Potts  External Field :  T = Tc ( =", t, "), H = 0", Direction, Value[0], "X", Value[1], "^ (", Value[2], "~", Value[3], ')')
        
        elif(Mode == 'Critical'):
            print(q, "- state ", D, "D  Potts  Critical :  T = Tc ( =", t, "), H = 0")
    
    elif(Status >= 0):
        if(Mode == 'T'):
            print("T =", round(Coupling_Parameter[Status][0], TRound), "= Tc ( =", t, ")", Direction, Value[0], "X", Value[1], "^", Value[2] + Status, '\n')
        elif(Mode == 'H'):
            print("H =", round(Coupling_Parameter[Status][1], HRound), "= 0", Direction, Value[0], "X", Value[1], "^", Value[2] + Status, '\n')

In [9]:
import time
import concurrent.futures

begin_time = time.time()

print("Start")
print("------------------------------------------------------------------------------------------------")
print(q, "- state ", D, "D  Potts Model  ( Size = ", end = "")
Size_Print()
print("Ensemble =", Ensemble, " Iteration =", Iteration, " Collect =", Collect, " Initial :", Initial)
Condition_Print(-1)
print("Data export | Normal :", Export[0], " Corr :", Export[1], " MCRG :", Export[2])
print("------------------------------------------------------------------------------------------------")

for num in range(len(Coupling_Parameter)):
    Parameter = Coupling_Parameter[num]
    T, H, t = Parameter[0], Parameter[1], (Parameter[0] - Tc) / Tc
    T, H, t = round(T, TRound), round(H, HRound), round(t, TRound)

    print("T =", T, " |  t = (T-Tc)/Tc =", t, " |  H =", H)
    Condition_Print(num)

    Data_Normal = [[] for _ in range(5)] # Order Parameter, Susceptibility, Binder Cumulant, Energy, Specific Heat
    Data_Corr = [[] for _ in range(Irreducible)] # Correlation Function
    Data_MCRG = [[] for _ in range(2)] # yh, yt
    
    Number = 1    
    t1 = time.time()
    
    with concurrent.futures.ProcessPoolExecutor(max_workers = 8) as executor:
        futures = [executor.submit(Record, Parameter) for _ in range(Ensemble)]
        
        for future in concurrent.futures.as_completed(futures):
            if Number < 10 : print(' ', Number, '/', Ensemble, f'completed: {int(time.time()-t1)}s')
            else: print('', Number, '/', Ensemble, f'completed: {int(time.time()-t1)}s')          
            Number = Number + 1
            
            Ensemble_Normal, Ensemble_Corr, Ensemble_MCRG = future.result()
            
            for i in range(5):
                Data_Normal[i].append(Ensemble_Normal[i])            
            for i in range(Irreducible):
                Data_Corr[i].append(Ensemble_Corr[i])            
            for i in range(2):
                Data_MCRG[i].append(Ensemble_MCRG[i])

    Data_Normal = [np.average(Data_Normal[i]) for i in range(5)]
    Data_Corr = [np.average(Data_Corr[i]) for i in range(Irreducible)]
    Data_MCRG = [np.average(Data_MCRG[i]) for i in range(2)]

    ExportData([Data_Normal, Data_Corr, Data_MCRG], Parameter, int(time.time() - t1))

    print("\nSub Total :  Time =", int(time.time() - t1), "s     T =", T, " |  t = (T-Tc)/Tc =", t, " |  H =", H)
    print("------------------------------------------------------------------------------------------------")
    
    del Parameter, T, H, t, Number, t1, Data_Normal, Data_Corr, Data_MCRG
    gc.collect()    

print(q, "- state ", D, "D  Potts Model ' ( Size = ", end = "")
Size_Print()
print("Ensemble =", Ensemble, " Iteration =", Iteration, " Collect =", Collect, " Initial :", Initial)
Condition_Print(-1)
print("Total :  Time =", int(time.time() - begin_time), "s")
print("------------------------------------------------------------------------------------------------")
print("Finish")

Start
------------------------------------------------------------------------------------------------
3 - state  2 D  Potts Model  ( Size = 64 x 64 )
Ensemble = 64  Iteration = 1024  Collect = 1024  Initial : Uniform
3 - state  2 D  Potts  Temperature :  H = 0, T = Tc ( = 1.005 ) + 0.0001 X 2 ^ ( 3 ~ 11 )
Data export | Normal : False  Corr : True  MCRG : False
------------------------------------------------------------------------------------------------
T = 1.006  |  t = (T-Tc)/Tc = 0.001  |  H = 0
T = 1.006 = Tc ( = 1.005 ) + 0.0001 X 2 ^ 3 

  1 / 64 completed: 947s
  2 / 64 completed: 953s
  3 / 64 completed: 954s
  4 / 64 completed: 955s
  5 / 64 completed: 956s
  6 / 64 completed: 959s
  7 / 64 completed: 964s
  8 / 64 completed: 965s
  9 / 64 completed: 1879s
 10 / 64 completed: 1883s
 11 / 64 completed: 1896s
 12 / 64 completed: 1898s
 13 / 64 completed: 1898s
 14 / 64 completed: 1903s
 15 / 64 completed: 1906s
 16 / 64 completed: 1914s
 17 / 64 completed: 2822s
 18 / 64 comp

In [ ]:
print("Start")
print("---------------------------------------------------------------------------------------------------------------------------------------")
print(q, "- state ", D, "D  Potts Model  ( Size = ", end = "")
Size_Print()
print("Ensemble =", Ensemble, " Iteration =", Iteration, " Collect =", Collect, " Initial :", Initial)
Condition_Print(-1)
print("Data export | Normal :", Export[0], " Corr :", Export[1], " MCRG :", Export[2])
print("---------------------------------------------------------------------------------------------------------------------------------------")

begin = time.time()

for i in range(len(Coupling_Parameter)):
    Parameter = Coupling_Parameter[i]
    T, H, t = Parameter[0], Parameter[1], (Parameter[0] - Tc) / Tc
    T, H, t = round(T, TRound), round(H, HRound), round(t, TRound)

    print("T =", T, " |  t = (T-Tc)/Tc =", t, " |  H =", H)
    Condition_Print(i)
    print("\nList of Ensembles Completed :")

    start = time.time()
    
    Data_Normal = [[] for i in range(5)] # Order Parameter, Susceptibility, Binder Cumulant, Energy, Specific Heat
    Data_Corr = [[] for i in range(Irreducible)] # Correlation Function
    Data_MCRG = [[] for i in range(2)] # yh, yt

    for j in range(Ensemble):
        start_Ensemble = time.time()
        
        Ensemble_Normal, Ensemble_Corr, Ensemble_MCRG = Record(Parameter)        
        
        # Ensemble_Print(j, int(time.time() - start_Ensemble))
        
        for k in range(5):
            Data_Normal[k].append(Ensemble_Normal[k])
        
        for k in range(Irreducible):
            Data_Corr[k].append(Ensemble_Corr[k])
        
        for k in range(2):
            Data_MCRG[k].append(Ensemble_MCRG[k])

        Ensemble_Print(j, int(time.time() - start_Ensemble))
    
    Data_Normal = [np.average(Data_Normal[i]) for i in range(5)]
    Data_Corr = [np.average(Data_Corr[i]) for i in range(Irreducible)]
    Data_MCRG = [np.average(Data_MCRG[i]) for i in range(2)]
    
    Time = int(time.time() - start)
    print("\n")
    
    ExportData([Data_Normal, Data_Corr, Data_MCRG], Parameter, Time)
    
    print("\nSub Total :  Time =", Time, "s     T =", T, " |  t = (T-Tc)/Tc =", t, " |  H =", H)
    print("---------------------------------------------------------------------------------------------------------------------------------------")

finish = int(time.time() - begin)

print(q, "- state ", D, "D  Potts Model ' ( Size = ", end = "")
Size_Print()
print("Ensemble =", Ensemble, " Iteration =", Iteration, " Collect =", Collect, " Initial :", Initial)
Condition_Print(-1)
print("Total :  Time =", finish, "s")
print("---------------------------------------------------------------------------------------------------------------------------------------")
print("Finish")

In [ ]:
import time
import multiprocessing

def do_something():

    print('1초간 잠을 잡니다...')

    time.sleep(1)

    print('잠에서 깨었습니다...')

start = time.perf_counter()

processes = []

for _ in range(4):
    p = multiprocessing.Process(target = do_something) ## 각 프로세스에 작업을 등록
    p.start()
    processes.append(p)

for process in processes:
    process.join()

finish = time.perf_counter()

print(f'{round(finish-start,2)}초 만에 작업이 완료되었습니다.')

In [ ]:
import time
import concurrent.futures

def do_something(file, num):
    # print('1초간 잠을 잡니다...')
    time.sleep(1)
    
    Temp = []
    
    for i in range(num + 1):
        Temp.append(i)
    
    file.append(Temp)
    
    return Temp

file = []

# do_something(file, 1)
# do_something(file, 2)

# print(file)

if __name__ == '__main__':
    start = time.perf_counter()

    file = []
    num = 1

    with concurrent.futures.ProcessPoolExecutor(max_workers = 4) as executor:
        futures = [executor.submit(do_something, file, i) for i in range(16)]
        
        for future in concurrent.futures.as_completed(futures):
            print(future.result())
            
            finish = time.perf_counter()
            print(num, f'번 {round(finish-start,2)}초 만에 작업이 완료되었습니다.')
            
            num = num + 1
    
    finish = time.perf_counter()

    print(f'{round(finish-start,2)}초 만에 작업이 완료되었습니다.')
    print(sum)

[0]
1 번 1.04초 만에 작업이 완료되었습니다.
[0, 1]
2 번 1.04초 만에 작업이 완료되었습니다.
[0, 1, 2]
3 번 1.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3]
4 번 1.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4]
5 번 2.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4, 5]
6 번 2.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4, 5, 6]
7 번 2.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4, 5, 6, 7]
8 번 2.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4, 5, 6, 7, 8]
9 번 3.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
10 번 3.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
11 번 3.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
12 번 3.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
13 번 4.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
14 번 4.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
15 번 4.04초 만에 작업이 완료되었습니다.
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
16 번 4.04초 만에 작업이 완료되었습니다.
4.05초 만에 작업이 완료되었습니다.
0


In [ ]:
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer

# normal function to run on cpu
def func(a):								
	for i in range(10000000):
		a[i]+= 1	

# function optimized to run on gpu
@jit(target_backend='cuda')						
def func2(a):
	for i in range(10000000):
		a[i]+= 1
if __name__=="__main__":
	n = 10000000							
	a = np.ones(n, dtype = np.float64)
	
	start = timer()
	func(a)
	print("without GPU:", timer()-start)	
	
	start = timer()
	func2(a)
	print("with GPU:", timer()-start)

without GPU: 1.1898531890037702
with GPU: 0.5999958040047204


In [ ]:
temp = []

for i in range(4):
    temp.append(i)

print(temp)

d = dir()

print(dir)

[0, 1, 2, 3]
<built-in function dir>
